In [92]:
import pandas
pandas.options.mode.chained_assignment = None

import statsmodels
from statsmodels.stats.multitest import fdrcorrection



In [93]:
# These name variables should be changed depending on what you're running. 

# Load the "non_spatial_eqtls.txt" output file from CoDeS3D (this file can be found under the /data folder)
# This output file is produced when you run CoDeS3D with the flags : --multi-test tissue --eqtl-project DICE --non-spatial
# Example : codes3d.py -s IV_SNPs.txt -o output_path --multi-test tissue --eqtl-project DICE --non-spatial --gene-list MR_causal_genes.txt

CoDeS3D_output = 'non_spatial_eqtls.txt'
DICE_eqtls = pandas.read_csv(CoDeS3D_output, sep='\t')


In [35]:

DICE_eqtls.head()

,snp,variant_id,gene,gencode_id,tissue,pval,b,b_se,snp_chrom,snp_locus,maf,gene_chrom,gene_start,gene_end
0,rs151174,chr16_28496748_C_T_b38,SLC25A20,ENSG00000178537.9,B_NAIVE,0.023246,-0.426667,0.183636,chr16,28496748,0.241573,chr3,48856931,48898993
1,rs40834,chr16_28499072_C_T_b38,SLC25A20,ENSG00000178537.9,B_NAIVE,0.013657,-0.407102,0.160655,chr16,28499072,0.421348,chr3,48856931,48898993
2,rs231977,chr16_28530851_T_C_b38,SLC25A20,ENSG00000178537.9,B_NAIVE,0.050608,-0.325098,0.163273,chr16,28530851,0.455056,chr3,48856931,48898993
3,rs151231,chr16_28571528_A_G_b38,SLC25A20,ENSG00000178537.9,B_NAIVE,0.142448,-0.254082,0.171162,chr16,28571528,0.466292,chr3,48856931,48898993
4,rs2927608,chr5_96916728_G_A_b38,SLC25A20,ENSG00000178537.9,B_NAIVE,0.087071,0.304883,0.175534,chr5,96916728,0.421348,chr3,48856931,48898993


In [11]:
# The 'pval' column from DICE_eqtls dataframe have not been FDR corrected

# First make an empty column called "adj_pval"
DICE_eqtls['adj_pval']=''

# A list of all DICE immune cell types
cell_types = list(DICE_eqtls['tissue'].unique())

# Run this code for FDR correction PER IMMUNE CELL TYPE
for x in cell_types:
    dataY = DICE_eqtls[DICE_eqtls['tissue'] == x]
    fdr = fdrcorrection(dataY['pval'])
    dataY['adj_pval']=fdr[1]
    DICE_eqtls.update(dataY['adj_pval'])


In [12]:
DICE_eqtls.head()

,snp,variant_id,gene,gencode_id,tissue,pval,b,b_se,snp_chrom,snp_locus,maf,gene_chrom,gene_start,gene_end,adj_pval
0,rs151174,chr16_28496748_C_T_b38,SLC25A20,ENSG00000178537.9,B_NAIVE,0.023246,-0.426667,0.183636,chr16,28496748,0.241573,chr3,48856931,48898993,0.577007
1,rs40834,chr16_28499072_C_T_b38,SLC25A20,ENSG00000178537.9,B_NAIVE,0.013657,-0.407102,0.160655,chr16,28499072,0.421348,chr3,48856931,48898993,0.528262
2,rs231977,chr16_28530851_T_C_b38,SLC25A20,ENSG00000178537.9,B_NAIVE,0.050608,-0.325098,0.163273,chr16,28530851,0.455056,chr3,48856931,48898993,0.660867
3,rs151231,chr16_28571528_A_G_b38,SLC25A20,ENSG00000178537.9,B_NAIVE,0.142448,-0.254082,0.171162,chr16,28571528,0.466292,chr3,48856931,48898993,0.728296
4,rs2927608,chr5_96916728_G_A_b38,SLC25A20,ENSG00000178537.9,B_NAIVE,0.087071,0.304883,0.175534,chr5,96916728,0.421348,chr3,48856931,48898993,0.69529


In [16]:
# Subset the dataframe to only show rows where adj_pval is less than 0.05!

significant_DICE_eqtls = DICE_eqtls[DICE_eqtls['adj_pval'] <= 0.05]


In [14]:
significant_DICE_eqtls.head()

,snp,variant_id,gene,gencode_id,tissue,pval,b,b_se,snp_chrom,snp_locus,maf,gene_chrom,gene_start,gene_end,adj_pval
108,rs12153855,chr6_32107027_T_C_b38,SLC25A20,ENSG00000178537.9,TREG_MEMORY,4.616283e-05,0.985507,0.225654,chr6,32107027,0.102273,chr3,48856931,48898993,0.00618
109,rs13211318,chr6_32134903_A_C_b38,SLC25A20,ENSG00000178537.9,TREG_MEMORY,4.616283e-05,0.985507,0.225654,chr6,32134903,0.102273,chr3,48856931,48898993,0.00618
1248,rs2927608,chr5_96916728_G_A_b38,ERAP2,ENSG00000164308.16,B_NAIVE,4.206857e-24,1.242393,0.078676,chr5,96916728,0.421348,chr5,96875939,96919716,0.0
1289,rs2927608,chr5_96916728_G_A_b38,ERAP2,ENSG00000164308.16,TH1,8.069903e-29,1.399903,0.066567,chr5,96916728,0.413580,chr5,96875939,96919716,0.0
1329,rs2927608,chr5_96916728_G_A_b38,ERAP2,ENSG00000164308.16,TREG_MEMORY,4.095775e-31,1.249057,0.058417,chr5,96916728,0.409091,chr5,96875939,96919716,0.0


# Determining the risk allele regulatory direction

In [37]:
# These name variables should be changed depending on what you're running. 

#load the MR result table (same as Supplementary Table 2)
MR = 'significant_MR_results.tsv'
MR_result = pandas.read_csv(MR, sep='\t')


# Load the "harmonised_data" output from the MR_analysis
H_D = 'harmonised_data.tsv'
harmonised_data = pandas.read_csv(H_D, sep='\t')

In [41]:
# Subset the "harmonised_data" dataframe to only show rows where the 'exposure' column is also in 'MR_result' dataframe
significant_exposures = harmonised_data[harmonised_data['exposure'].isin(MR_result['exposure'])]


In [42]:
significant_exposures

,SNP,effect_allele.exposure,other_allele.exposure,effect_allele.outcome,other_allele.outcome,beta.exposure,beta.outcome,eaf.exposure,eaf.outcome,remove,...,se.exposure,pval.exposure,exposure,mr_keep.exposure,pval_origin.exposure,id.exposure,data_source.exposure,action,mr_keep,samplesize.outcome
4654,rs10484565,A,G,A,G,-0.288260,-0.076131,0.072388,0.077472,False,...,0.049816,1.158328e-08,HLA-DOB,True,reported,fHBCb8,textfile,2,True,NaN
4655,rs4148876,A,G,A,G,0.968278,-0.313643,0.069403,0.063494,False,...,0.037590,4.186701e-99,HLA-DOB,True,reported,fHBCb8,textfile,2,True,NaN
4656,rs9276856,T,C,T,C,-0.118424,0.033351,0.250746,0.278973,False,...,0.030291,1.029947e-04,HLA-DOB,True,reported,fHBCb8,textfile,2,True,NaN
6683,rs10807029,A,G,A,G,-0.055349,-0.145349,0.388060,0.390670,False,...,0.013878,7.480762e-05,HSP90AB1,True,reported,uArlIh,textfile,2,True,NaN
16066,rs12153855,C,T,C,T,0.334803,-0.396516,0.108955,0.095351,False,...,0.046982,2.971588e-12,FKBPL,True,reported,kGiuF3,textfile,2,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32738,rs9378423,A,T,A,T,0.069798,0.037607,0.494776,0.477385,False,...,0.019068,2.739321e-04,TRIM26,True,reported,Pah4U5,textfile,2,False,NaN
32741,rs9296059,C,A,C,A,0.174077,0.288198,0.097015,0.084425,False,...,0.034097,4.440726e-07,HLA-DMB,True,reported,QXEKBz,textfile,2,True,NaN
32784,rs9368609,T,C,T,C,0.688787,0.273383,0.238806,0.259859,False,...,0.050248,2.115403e-37,IFITM4P,True,reported,MnRJUk,textfile,2,True,NaN
32832,rs9469220,A,G,A,G,-0.073349,0.285140,0.485821,0.495609,False,...,0.018099,5.731951e-05,HLA-DRB1,True,reported,uWd2Nx,textfile,2,True,NaN


In [43]:
# "effect_allele" of each SNP always corresponds to "alt" allele in Grch38 human reference genome
# This means that for each SNP, if the "beta.outcome" is negative, that means that the "alt allele" of the SNP is the PROTECTIVE ALLELE
# So, if we want to get the beta of the eQTL in the RISK ALLELE direction we have to SWITCH THE BETA (beta * -1)

SNPs_alt_is_risk = list(significant_exposures[significant_exposures['beta.outcome']>0]['SNP'])
SNPs_alt_is_not_risk = list(significant_exposures[significant_exposures['beta.outcome']<0]['SNP'])

# Making input matrix for hierarchical clustering (used to make Figure 2)

In [94]:
#lets make IV SNP-Gene pairs dataframe
IV_SNP_gene_pairs = significant_exposures[['SNP','exposure']]

#Lets make a new column called "lead_SNP-Gene"
IV_SNP_gene_pairs['IV SNP - exposure gene'] = IV_SNP_gene_pairs[['SNP','exposure']].agg('--'.join, axis=1)


In [95]:
IV_SNP_gene_pairs

,SNP,exposure,IV SNP - exposure gene
4654,rs10484565,HLA-DOB,rs10484565--HLA-DOB
4655,rs4148876,HLA-DOB,rs4148876--HLA-DOB
4656,rs9276856,HLA-DOB,rs9276856--HLA-DOB
6683,rs10807029,HSP90AB1,rs10807029--HSP90AB1
16066,rs12153855,FKBPL,rs12153855--FKBPL
...,...,...,...
32738,rs9378423,TRIM26,rs9378423--TRIM26
32741,rs9296059,HLA-DMB,rs9296059--HLA-DMB
32784,rs9368609,IFITM4P,rs9368609--IFITM4P
32832,rs9469220,HLA-DRB1,rs9469220--HLA-DRB1


In [96]:
# Lets make an empty dataframe to accomodate the input matrix for hierarchical clustering with pheatmap R package 
column_names = ['SNP', 'exposure', 'IV SNP - exposure gene','B_NAIVE',
 'CD4_NAIVE',
 'CD4_N_STIM',
 'CD8_NAIVE',
 'CD8_N_STIM',
 'CLASSICAL_MONOCYTES',
 'NK_CD16POS',
 'NONCLASSICAL_MONOCYTES',
 'TFH',
 'TH1',
 'TH17',
 'TH1_17',
 'TH2',
 'TREG_MEMORY',
 'TREG_NAIVE']

DICE_matrix = pandas.DataFrame(columns = column_names)
DICE_matrix

,SNP,exposure,IV SNP - exposure gene,B_NAIVE,CD4_NAIVE,CD4_N_STIM,CD8_NAIVE,CD8_N_STIM,CLASSICAL_MONOCYTES,NK_CD16POS,NONCLASSICAL_MONOCYTES,TFH,TH1,TH17,TH1_17,TH2,TREG_MEMORY,TREG_NAIVE


In [97]:
#Lets fill the first three columns with the values from the dataframe "IV_SNP_gene_pairs"
DICE_matrix[['SNP','exposure','IV SNP - exposure gene']] = IV_SNP_gene_pairs

In [98]:
DICE_matrix

,SNP,exposure,IV SNP - exposure gene,B_NAIVE,CD4_NAIVE,CD4_N_STIM,CD8_NAIVE,CD8_N_STIM,CLASSICAL_MONOCYTES,NK_CD16POS,NONCLASSICAL_MONOCYTES,TFH,TH1,TH17,TH1_17,TH2,TREG_MEMORY,TREG_NAIVE
4654,rs10484565,HLA-DOB,rs10484565--HLA-DOB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4655,rs4148876,HLA-DOB,rs4148876--HLA-DOB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4656,rs9276856,HLA-DOB,rs9276856--HLA-DOB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6683,rs10807029,HSP90AB1,rs10807029--HSP90AB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16066,rs12153855,FKBPL,rs12153855--FKBPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32738,rs9378423,TRIM26,rs9378423--TRIM26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32741,rs9296059,HLA-DMB,rs9296059--HLA-DMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32784,rs9368609,IFITM4P,rs9368609--IFITM4P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32832,rs9469220,HLA-DRB1,rs9469220--HLA-DRB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
#Fill all other cell with 0s !
DICE_matrix.iloc[:,3:] = 0

In [100]:
#Now lets fill the cells with +Absolute beta  or - absolute beta depending if the risk allele is the alt or the ref!


for index, row in DICE_matrix.iterrows():
    for cell in list(significant_DICE_eqtls['tissue'].unique()):
        if row[0] in SNPs_alt_is_risk:
            my_filter = (significant_DICE_eqtls['gene'] == row[1]) & (significant_DICE_eqtls['tissue'] == cell) & (significant_DICE_eqtls['snp'] == row[0])
            try:
                DICE_matrix.loc[index, cell] += list(significant_DICE_eqtls[my_filter]['b'])[0]
            except:
                DICE_matrix.loc[index, cell] += 0
        
        elif row[0] in SNPs_alt_is_not_risk:
            my_filter = (significant_DICE_eqtls['gene'] == row[1]) & (significant_DICE_eqtls['tissue'] == cell) & (significant_DICE_eqtls['snp'] == row[0])
            try:
                DICE_matrix.loc[index, cell] += -list(significant_DICE_eqtls[my_filter]['b'])[0]
            except:
                DICE_matrix.loc[index, cell] += 0
                

In [101]:
DICE_matrix

,SNP,exposure,IV SNP - exposure gene,B_NAIVE,CD4_NAIVE,CD4_N_STIM,CD8_NAIVE,CD8_N_STIM,CLASSICAL_MONOCYTES,NK_CD16POS,NONCLASSICAL_MONOCYTES,TFH,TH1,TH17,TH1_17,TH2,TREG_MEMORY,TREG_NAIVE
4654,rs10484565,HLA-DOB,rs10484565--HLA-DOB,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4655,rs4148876,HLA-DOB,rs4148876--HLA-DOB,0,-1.497054,-1.57213,-1.655267,-1.556433,-1.544339,-1.554208,-1.495003,-1.468333,-1.619392,-1.450041,-1.538766,-1.609371,-1.512335,-1.523318
4656,rs9276856,HLA-DOB,rs9276856--HLA-DOB,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6683,rs10807029,HSP90AB1,rs10807029--HSP90AB1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16066,rs12153855,FKBPL,rs12153855--FKBPL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32738,rs9378423,TRIM26,rs9378423--TRIM26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32741,rs9296059,HLA-DMB,rs9296059--HLA-DMB,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32784,rs9368609,IFITM4P,rs9368609--IFITM4P,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
32832,rs9469220,HLA-DRB1,rs9469220--HLA-DRB1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [104]:
#Lets subset the dataframe to only show rows where column 3 and above contain at least ONE NON ZERO value !
DICE_matrix = DICE_matrix[(DICE_matrix.iloc[:,3:18]!=0).any(axis=1)]


In [105]:
#Lets save as csv. we will use this to make Figure 2 using pheatmap R package
DICE_matrix.to_csv('DICE_matrix.csv', index=False)